# RAG 시스템 - 문서 질의 및 응답 (Document Query & Response)

이 노트북은 저장된 문서에 대해 질문하고 LLM을 통해 답변을 생성하는 과정을 설명합니다.

## 주요 구성요소
1. 벡터 검색 (Vector Search)
2. LLM 연동 (Language Model Integration)
3. 응답 생성 (Response Generation)

## 사전 요구사항
- ChromaDB 서버 실행 중
- Ollama 서버 실행 중
- deepseek-r1:8b 모델 다운로드 완료

## 1. 필요한 라이브러리 임포트 및 설정

RAG 시스템의 질의-응답에 필요한 설정을 진행합니다:
- requests: Ollama API 호출
- Chroma: 벡터 데이터베이스
- 서버 설정 및 모델 지정

In [ ]:
import json
import requests
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Ollama 서버 설정
OLLAMA_HOST = "http://localhost:11434"
OLLAMA_MODEL = "deepseek-r1:8b"

# ChromaDB 설정
PERSIST_DIR = "./data"
COLLECTION_NAME = "rag_test"

## 2. 임베딩 모델 설정

문서 검색을 위한 임베딩 모델을 준비합니다:
- nomic-embed-text-v1 모델
- CPU 기반 처리
- 정규화된 임베딩

In [ ]:
def get_embeddings():
    """한국어에 최적화된 임베딩 모델 생성"""
    return HuggingFaceEmbeddings(
        model_name="nomic-ai/nomic-embed-text-v1",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

## 3. 벡터 스토어 로드

ChromaDB에 연결하여 저장된 문서에 접근합니다:
- 임베딩 모델 초기화
- 지정된 컬렉션 연결
- 에러 처리

In [ ]:
def load_vector_store():
    """ChromaDB에서 벡터 스토어 로드"""
    try:
        embedding_model = get_embeddings()
        vector_store = Chroma(
            persist_directory=PERSIST_DIR,
            embedding_function=embedding_model,
            collection_name=COLLECTION_NAME
        )
        return vector_store
    except Exception as e:
        print(f"❌ 벡터 스토어 로드 실패: {str(e)}")
        return None

## 4. Ollama API 호출

LLM 모델을 호출하여 응답을 생성합니다:
- 한글 응답 최적화
- 스트리밍 응답 처리
- 응답 정제 및 포맷팅

In [ ]:
def query_ollama(prompt):
    """Ollama API를 사용하여 LLM 모델 호출"""
    try:
        # 프롬프트에 한글 응답 요청 추가
        korean_prompt = f"""
다음 지시사항을 엄격히 따라 답변해주세요:

1. 반드시 한글로만 답변하세요.
2. 영어 단어는 모두 한글로 변환하세요 (예: API -> 에이피아이).
3. 특수문자나 한자는 절대 사용하지 마세요.
4. 간단명료하게 답변하세요.
5. 불필요한 설명이나 부연은 제외하세요.
6. 답변 전에 생각하는 과정을 보여주지 마세요.
7. 바로 결과만 보여주세요.

질문:
{prompt}

답변 형식:
[질문에 대한 답변만 작성]
"""
        response = requests.post(
            f"{OLLAMA_HOST}/api/generate",
            json={
                "model": OLLAMA_MODEL,
                "prompt": korean_prompt,
                "system": "당신은 한국어 전용 답변 도우미입니다."
            },
            stream=True
        )
        
        full_response = ""
        for line in response.iter_lines():
            if line:
                json_response = json.loads(line)
                if 'response' in json_response:
                    full_response += json_response['response']
        
        # 응답 정리
        full_response = full_response.replace('<think>\n', '').replace('</think>', '')
        if '[' in full_response:
            full_response = full_response.split(']')[-1].strip()
            
        return full_response.strip()

    except requests.exceptions.RequestException as e:
        return f"🚨 Ollama 연결 오류: {e}"

## 5. RAG 질의 실행

전체 RAG 프로세스를 실행하는 함수입니다:
1. 벡터 검색으로 관련 문서 찾기
2. 문서 컨텍스트 구성
3. 프롬프트 생성
4. LLM으로 답변 생성

In [ ]:
def run_rag_query(query):
    """벡터DB에서 질문에 대한 답변을 검색"""
    try:
        # 벡터 스토어 로드
        vector_store = load_vector_store()
        if not vector_store:
            return "벡터 스토어를 로드할 수 없습니다."

        # 유사도 검색 수행
        results = vector_store.similarity_search(query, k=3)
        
        # 검색된 문서 처리
        contexts = []
        for i, doc in enumerate(results, 1):
            contexts.append(f"문서 {i}:\n{doc.page_content}")
        
        context = "\n\n=== 다음 문서 ===\n\n".join(contexts)
        
        print(f"\n💾 벡터 DB 콜렉션 조회 성공")
        print(f"🔍 관련 문서 {len(results)}개 찾음")
        
        # 프롬프트 구성
        prompt = f"""역할: 당신은 주어진 문서들에서 모든 관련 정보를 찾아 종합적으로 답변하는 역할을 합니다.

문서 내용:
{context}

질문: {query}

중요 지침:
1. 모든 문서의 내용을 검토하여 관련된 정보를 모두 찾아주세요.
2. 각 문서의 정보를 종합하여 하나의 완성된 답변을 만들어주세요.
3. 문서에 없는 내용은 추가하지 마세요.
4. 외부 지식이나 추론은 하지 마세요.
5. 영어나 특수문자는 최소한으로 사용하세요.
6. 여러 문서의 정보가 있다면 모두 포함해서 답변해주세요.
7. 답변은 간결하면서도 포괄적이어야 합니다.
8. 문서에서 관련 내용을 찾을 수 없다면 '주어진 문서에서 관련 정보를 찾을 수 없습니다.'라고만 답변해주세요.

답변: """
        
        # Ollama로 답변 생성
        response = query_ollama(prompt)
        print(f"\n💬 답변: {response}\n")
        return response
    except Exception as e:
        print(f"❌ RAG 검색 중 오류 발생: {e}")
        return "⚠️ RAG 검색 오류"

## 6. 예제: 질문하기

실제 질문을 통해 RAG 시스템을 테스트합니다:
- 간단한 질문 예시
- 질문과 답변 출력
- 시스템 동작 확인

In [ ]:
# 예제: 질문하기
question = "한국의 봄은 어떤 특징이 있나요?"
answer = run_rag_query(question)
print(f"질문: {question}")
print(f"답변: {answer}")